# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [6]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [7]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [9]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [10]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/ll

In [11]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [14]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://tietoevry.com")
huggingface.links

['/en/',
 '/at/',
 '/ch/',
 '/cn/',
 '/cz/',
 '/de/',
 '/dk/',
 '/ee/',
 '/fi/',
 '/in/',
 '/lt/',
 '/lv/',
 '/no/',
 '/pl/',
 '/se/',
 '/ua/',
 '/en/contact-tietoevry/',
 '/en/services/',
 '/en/create/',
 '/en/create/',
 '/en/create/design-and-experience/',
 '/en/create/design-and-experience/',
 '/en/create/data-and-analytics-services/',
 '/en/create/data-and-analytics-services/',
 '/en/create/advisory-design-and-innovation/',
 '/en/create/advisory-design-and-innovation/',
 '/en/create/software-engineering-services/',
 '/en/create/software-engineering-services/',
 '/en/create/software-engineering-services/automotive-software-solutions/',
 '/en/create/software-engineering-services/automotive-software-solutions/',
 '#modal_0',
 '/en/tech-services/',
 '/en/tech-services/',
 '/en/tech-services/application-services/',
 '/en/tech-services/application-services/',
 '/en/tech-services/enterprise-applications/',
 '/en/tech-services/enterprise-applications/',
 '/en/tech-services/ai-data-analytic

In [18]:
get_links("https://tietoevry.com")

{'links': [{'type': 'about page',
   'url': 'https://www.tietoevry.com/en/about-us/'},
  {'type': 'careers page', 'url': 'https://www.tietoevry.com/en/careers/'},
  {'type': 'sustainability page',
   'url': 'https://www.tietoevry.com/en/sustainability/'},
  {'type': 'contact page',
   'url': 'https://www.tietoevry.com/en/contact-tietoevry/'},
  {'type': 'investor relations page',
   'url': 'https://www.tietoevry.com/en/investor-relations/'},
  {'type': 'newsroom page', 'url': 'https://www.tietoevry.com/en/newsroom/'},
  {'type': 'success stories page',
   'url': 'https://www.tietoevry.com/en/success-stories/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [19]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [20]:
print(get_all_details("https://tietoevry.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.tietoevry.com/en/about-us/'}, {'type': 'careers page', 'url': 'https://www.tietoevry.com/en/careers/'}, {'type': 'sustainability page', 'url': 'https://www.tietoevry.com/en/sustainability/'}, {'type': 'newsroom', 'url': 'https://www.tietoevry.com/en/newsroom/'}, {'type': 'contact page', 'url': 'https://www.tietoevry.com/en/contact-tietoevry/'}]}
Landing page:
Webpage Title:
Creating purposeful technology | Tietoevry
Webpage Contents:
noun_Email_707352
noun_917542_cc
Map point
Play
Untitled
Retweet
Group 3
Fill 1
Home
Search
Close
EN
Local sites
AT - Austria
CH- Switzerland
CN - China
CZ - Czech
DE - Germany
DK - Denmark
EE - Estonia
EN - Global
FI - Finland
IN - India
LT - Lithuania
LV - Latvia
NO - Norway
PL - Poland
SE - Sweden
UA - Ukraine
Contact us
Toggle navigation
Services
Services
What We Offer
Service hubs with tailored expertise to help you accelerate your business​
Toggle navigation
Tietoevry Create
Creating 

In [40]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [43]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown format.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 10,000 characters
    return user_prompt

In [44]:
get_brochure_user_prompt("Tietoevry", "https://www.tietoevry.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.tietoevry.com/en/about-us/'}, {'type': 'careers page', 'url': 'https://www.tietoevry.com/en/careers/'}, {'type': 'sustainability page', 'url': 'https://www.tietoevry.com/en/sustainability/'}, {'type': 'contact page', 'url': 'https://www.tietoevry.com/en/contact-tietoevry/'}, {'type': 'investor relations page', 'url': 'https://www.tietoevry.com/en/investor-relations/'}, {'type': 'newsroom page', 'url': 'https://www.tietoevry.com/en/newsroom/'}]}


'You are looking at a company called: Tietoevry\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown format.\nLanding page:\nWebpage Title:\nCreating purposeful technology | Tietoevry\nWebpage Contents:\nnoun_Email_707352\nnoun_917542_cc\nMap point\nPlay\nUntitled\nRetweet\nGroup 3\nFill 1\nHome\nSearch\nClose\nEN\nLocal sites\nAT - Austria\nCH- Switzerland\nCN - China\nCZ - Czech\nDE - Germany\nDK - Denmark\nEE - Estonia\nEN - Global\nFI - Finland\nIN - India\nLT - Lithuania\nLV - Latvia\nNO - Norway\nPL - Poland\nSE - Sweden\nUA\xa0- Ukraine\nContact us\nToggle navigation\nServices\nServices\nWhat We Offer\nService hubs with tailored expertise to help you accelerate your business\u200b\nToggle navigation\nTietoevry Create\nCreating novel solutions\nDesign and experience\nData, analytics and AI\nAdvisory and innovation\nSoftware engineering\nAutomotive software services\nCosimo  De Carlo\nManagin

In [45]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [46]:
create_brochure("Tietoevry", "https://tietoevry.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://tietoevry.com/en/about-us/'}, {'type': 'careers page', 'url': 'https://tietoevry.com/en/careers/'}, {'type': 'sustainability page', 'url': 'https://tietoevry.com/en/sustainability/'}, {'type': 'investor relations', 'url': 'https://tietoevry.com/en/investor-relations/'}, {'type': 'contact page', 'url': 'https://tietoevry.com/en/contact-tietoevry/'}]}


# Welcome to Tietoevry: Where Purpose Meets Pixels!

**Get Ready for Purposeful Technology**  
At Tietoevry, we believe in a world where technology is not just smart but also incredibly purposeful. We’re like the Swiss Army Knife of the tech world, providing you with solutions ranging from AI wizardry to banking magic, all while ensuring our planet is green and our workplace feels like a warm hug!

---

## 🎢 What Do We Do?

**Creating Novel Solutions**  
Whether you need a snazzy app to woo your customers or AI you can actually trust (no bots plotting world domination here), we’ve got it all! 🚀

- **Banking & Finance?** We’ve got your back with Banking as a Service. 
- **Need healthcare solutions?** Our Tietoevry Care services are the modern miracle you’ve been waiting for! 
- **Industry-focused?** From pulp to packaging, we digitally transform industries so they can thrive in a tech-savvy world.

---

## 🌍 Global Presence, Local Flavor

We’re not just your average tech company; we’re a colorful patchwork of locations!  
From Austria to Australia (okay, we wish), and every country in Europe, we've got our hands in many pots, including China, India, and Ukraine! It’s like a tech-themed world tour, minus the jet lag.

---

## 🚀 Join Us in Building an AI-Powered Future

AI is our buddy (and yours too!). We focus on responsible AI that empowers human potential without the drama of rogue algorithms.  
Join us on our quest, and who knows, maybe you'll create the next AI that actually understands your cat's meows!

---

## 🤝 Our Customers

Our clients range from ambitious fintech startups to sleepy industrial giants that woke up in the digital age. And we’re proud to say our tech helps them all stay ahead of the curve and avoid the digital dinosaur extinction!

---

## 🎉 Company Culture & Careers: Where Fun Meets Function

At Tietoevry, you'll find work-life harmony that’s more balanced than a tightrope walker during a yoga class. Whether you are a strategist, coder, or a digital oracle, there’s a cozy spot for you in our tech kingdom. 

**Fun Fact:** Our meetings often involve more puns than PowerPoints! Who said techies don’t know how to have a good time?

---

## 🌱 Sustainability: We’re Green, Not Just in Code

We take our commitment to sustainability as seriously as we take our caffeine (which, let’s be honest, is very serious!). From climate action to inclusivity, we believe in tech that gives back. Because saving the planet isn’t just a slogan, it’s a Tietoevry lifestyle!

---

## 🎈 Why Tietoevry?

- **Diverse Teams:** We ensure our teams look like a rainbow, bringing perspectives that help us innovate.
- **Supportive Environment:** We actually encourage you to bring your dog to work (just make sure it doesn’t chew on the cables).
- **Career Growth:** Sky's the limit! Well, unless you work in cloud computing—then the cloud's the limit!

---

## 📞 Let’s Connect!

If you’re ready to take the plunge into purposeful technology (or you just want to see pictures of our office dogs), reach out to us! We promise to bring both innovation and joy to your inbox.

**Email:** [hello@tietoevry.com](mailto:hello@tietoevry.com)  
**Website:** [www.tietoevry.com](http://www.tietoevry.com) 

Join us at Tietoevry, where every byte counts and every laugh echoes in the halls! 🎉

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [48]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [49]:
stream_brochure("Tietoevry", "https://tietoevry.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.tietoevry.com/en/about-us/'}, {'type': 'careers page', 'url': 'https://www.tietoevry.com/en/careers/'}, {'type': 'sustainability page', 'url': 'https://www.tietoevry.com/en/sustainability/'}]}


# Welcome to Tietoevry - Where Purposeful Technology Meets a Dash of Humor!

---

## **What is Tietoevry?**

At Tietoevry, we’re serious about making technology that matters. But hey, a little fun never hurt anyone! From enhancing banking services to modernizing healthcare, we’re on a mission to create **purposeful technology** that looks good, works great, and maybe even tells a joke or two (just not during your loan application process).

---

## **Our Services - We Do All the Tech-y Things!**

### **1. Tietoevry Create**
We make technology look *so* easy, it's like teaching a cat how to use a laptop (don’t try this at home). Our specialties include:
- Novel solutions
- Design & experience
- Data, analytics, and AI (no magic wands required!)

### **2. Tietoevry Tech Services**
Need your enterprise IT modernized? We've got the tools. And the talent. And the snacks. 
- Next-gen application services
- Cloud and infrastructure (heavenly, right?)
- Security services (because who likes hackers? Not us!)

### **3. Tietoevry Banking**
Banking as easy as pie (which we also like)! 
- Transaction banking (NOT the same as transaction dancing!)
- Financial crime prevention (we’re the financial superheroes you didn’t know you needed)

### **4. Tietoevry Care**
Modernizing Nordic health and care while keeping a sense of humor is our motto! 
- Data and AI for health improvements
- Because who wouldn’t want a fit and funny healthcare system?

---

## **Our Company Culture - We Keep It Real!**

### **Join the Fun!**
We believe in a workplace that’s as diverse and vibrant as a box of chocolates (or your favorite candy — no judgment here). Whether you’re a coder, analyst, or a self-proclaimed "future enthusiast," here’s why you’ll love us:
- **Collaboration**: Like peanut butter and jelly! 
- **Innovation**: Where ideas flourish. Sorta like the plants you forgot to water last week.
- **Flexibility**: We understand that your best ideas might come at 9 PM… or 2 AM, or wherever you are (crying into coffee). 

---

## **Customers We Serve**
You might see us working hard for:
- Financial Institutions (who needs all that stress?)
- Healthcare & Welfare (because health jokes are essential!)
- Energy & Utilities (because saving the planet is on everyone’s to-do list)

---

## **Careers - Your Future Awaits!**

Looking to work somewhere you can make a difference and maybe *shift* the universe a bit? Look no further! 
- 🌟 **Student Programs**: Jump-start your career with us!
- 🧑‍💻 **Job Openings**: Strategists, coders, analysts—all are welcome!
- 👩‍🏫 **A Day in the Life**: Spoiler alert: It involves amazing coworkers, tech-filled spaces, and possibly a cat video or two.

---

## **Investors - Making Dollars and Sense!**

Curious about investing with us? Let’s turn those coins into community-building efforts! We promise transparency. Our reports may not be as thrilling as a Marvel movie, but they sure tell a captivating story about our growth! 

## **Contact Us – Let’s Connect!**

Whether you want to learn more about our services, join our team, or roll out potential investment opportunities, don’t be shy—reach out! We promise we won’t bite...unless it’s donut day! 

📞 [Contact Us](mailto:news@tietoevry.com)

---

**Tietoevry** - Creating Technology that Matters, One Laugh at a Time! 🤖✨

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>